In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import os
import csv
from census import Census
from us import states
from pprint import pprint
# Census API Key
from config import api_key
# c = Census(api_key, year=2000)
# c = Census(api_key, year=2010)
# census_data = c.acs5.get(('NAME', 'B25034_010E'),
#                        {'for': 'state:{}'.format(states.MD.fips)})
# census_data = c.sf1.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
#                           "B19301_001E",
#                           "B17001_002E",
#                           "B23025_005E"), {'for': 'state:*'})
#
# P001001	Total population
#

In [2]:
# https://api.census.gov/data.html
# Pulling 2000 population from sf3 Census Summary file 3
#
base_url = "https://api.census.gov/data/2000/sf3?"
query = f"{base_url}get=P001001,NAME&for=county:*&key={api_key}"
resp = requests.get(query).json()
print(len(resp))

3220


In [3]:
p2000_df = pd.DataFrame(resp, columns=['Population','County Name','State Code','County Code'])
# To delete first row which have column names.
pop2000_df = p2000_df.iloc[1:]
print(len(pop2000_df))
pop2000_df.head()

3219


,Population,County Name,State Code,County Code
1,43671,Autauga County,01,001
2,140415,Baldwin County,01,003
3,29038,Barbour County,01,005
4,20826,Bibb County,01,007
5,51024,Blount County,01,009


In [8]:
pop2000_df.to_csv(r'../Resources/population_2000_api_data.csv', index=False, header=True)

In [ ]:
# pprint(resp)

In [ ]:
# r = pop2010_df.loc[result['County Name'] == 'Autauga County']
# r

In [4]:
# google search with census P001001
# https://api.census.gov/data/2010/dec/sf1?get=P001001,NAME&for=county:*&key=YOUR_KEY_GOES_HERE
# For the year 2010
#
base_url = "https://api.census.gov/data/2010/dec/sf1?"
query = f"{base_url}get=P001001,NAME&for=county:*&key={api_key}"
resp = requests.get(query).json()
print(len(resp))

3222


In [ ]:
# #     result = pd.DataFrame({"Population":resp[x][0],
# #                            "County":cc[0],
# #                            "State":cc[1],
# #                            "StateCode":resp[x][2],
# #                            "CountyCode":resp[x][3]
# #                           })

In [6]:
#  Cleaning the data by spliting the columns for 2010 census data...
#
col1 = []
col2 = []
col3 = []
col4 = []
col5 = []
for x in range(len(resp)):
    if x > 0:
        col1.append(resp[x][0])
        cc = str(resp[x][1]).split(",")
        col2.append(cc[0])
        col3.append(cc[1])
        col4.append(resp[x][2])
        col5.append(resp[x][3])
result = list(zip(col1,col2,col3,col4,col5))
pop2010_df = pd.DataFrame(result, columns=['Population','County Name','State','State Code','County Code'])
print(len(pop2010_df))
pop2010_df.head()

3221


,Population,County Name,State,State Code,County Code
0,125744,Sebastian County,Arkansas,05,131
1,17058,Sevier County,Arkansas,05,133
2,17264,Sharp County,Arkansas,05,135
3,12394,Stone County,Arkansas,05,137
4,41639,Union County,Arkansas,05,139


In [7]:
pop2010_df.to_csv(r'../Resources/population_2010_api_data.csv', index=False, header=True)

In [ ]:
# pprint(resp)

In [ ]:
# # https://api.census.gov/data/2005/acs/acs1/profile?get=NAME,DP01_0001E&for=county:*&key=YOUR_KEY_GOES_HERE
# #
# # For the year 2005
# #
# base_url = "https://api.census.gov/data/2005/acs/acs1/profile?"
# query = f"{base_url}get=NAME,DP01_0001E&for=county:*&key={api_key}"
# resp = requests.get(query).json()
# print(len(resp))
# # It's bringing only 788 rows, ignoring this...

In [ ]:
#  For the year 2004
# https://api.census.gov/data/2008/acs/acs3?get=NAME,B00001_001E&for=county:*&in=state:*&key=YOUR_KEY_GOES_HERE
# 
# these are samples

In [ ]:
temp_df = pd.read_csv("../Resources/2000.csv")
temp_df

In [ ]:
temp_df.columns = ['ghcnd_station_code','date','descr','value','col5','col6','unit','col7']
temp_df.head()
# temp_df["date"].unique
temp_df

In [ ]:
w2000_df = temp_df.loc[temp_df["date"] == 20001106]
w2000_df

In [ ]:
ghcnd_df = pd.read_csv("../Resources/ghcnd-stations.csv")
ghcnd_df.columns = ['ghcnd_station_code', 'Lat', 'Lng', 'Elevation', 'Name','Flg', 'Wmoid']
ghcnd_df['Name'] = ghcnd_df['Name'].str.strip()
ghcnd_df['StName'] = ghcnd_df['Name'].str[:2]
ghcnd_df

In [ ]:
w2000 = pd.merge(w2000_df, ghcnd_df, on='ghcnd_station_code')
w2000

In [ ]:
weather_2000 = w2000[['date', 'descr', 'value', 'unit', 'Lat', 'Lng', 'StName', 'Name']]
weather_2000

In [ ]:
# # Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# # See: https://github.com/CommerceDataService/census-wrapper for library documentation
# # See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B25034_010E": "Population",
                                      "B25034_010E": "Median Age",
                                      "B25034_010E": "Household Income",
                                      "B25034_010E": "Per Capita Income",
                                      "B25034_010E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)
# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]

# Visualize
print(len(census_pd))
census_pd

In [ ]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
# census_pd.to_csv("census_data.csv", encoding="utf-8", index=False)